In [1]:
pip install optuna scikit-learn

     |████████████████████████████████| 296kB 11.3MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 1.1MB 27.3MB/s 
     |████████████████████████████████| 112kB 55.3MB/s 
     |████████████████████████████████| 143kB 59.3MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 481kB 46.5MB/s 
  Created wheel for alembic: filename=alembic-1.5.3-py2.py3-none-any.whl size=155547 sha256=220a85c8b44c2fe6fbf2a110a808973f2af4a882782f04dde5947a86ac9e09dd
  Stored in directory: /root/.cache/pip/wheels/7f/ed/1b/a66a0cbca75fd3e374bd5cc60c443e5675f10ef1f1f78ec31d
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=be4182b2341ae50a1863d233e37473b50f06be14f4c7701db4cb24b83a036918
  Stored in directory: /root/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sh

In [ ]:
from functools import partial

import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn import datasets


def objective(X, y, trial):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear','rbf','poly','sigmoid']),
        'C': trial.suggest_loguniform('C', 1e-3, 1e+3),
        'gamma': trial.suggest_loguniform('gamma', 1e-3, 1e+3)
    }
    model = SVC(**params)

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1000)
    scores = cross_validate(model, X=X, y=y, cv=kf)
    return 1.0 - scores['test_score'].mean()


dataset = datasets.load_digits()
X, y = dataset.data, dataset.target
f = partial(objective, X, y)
study = optuna.create_study()
study.optimize(f, n_trials=1000)
print('params:', study.best_params)

[I 2021-02-01 08:05:08,135] A new study created in memory with name: no-name-221d024f-dd93-43e3-bd9b-72c8e0153059
[I 2021-02-01 08:05:09,844] Trial 0 finished with value: 0.8987186629526462 and parameters: {'kernel': 'sigmoid', 'C': 0.6875095858268175, 'gamma': 0.0215748827174723}. Best is trial 0 with value: 0.8987186629526462.
[I 2021-02-01 08:05:11,522] Trial 1 finished with value: 0.8987186629526462 and parameters: {'kernel': 'sigmoid', 'C': 0.00160213012133526, 'gamma': 6.6778144699006114}. Best is trial 0 with value: 0.8987186629526462.
[I 2021-02-01 08:05:11,826] Trial 2 finished with value: 0.011685236768802176 and parameters: {'kernel': 'poly', 'C': 2.4912341110852383, 'gamma': 0.011449213471849814}. Best is trial 2 with value: 0.011685236768802176.
[I 2021-02-01 08:05:12,097] Trial 3 finished with value: 0.028384401114206148 and parameters: {'kernel': 'linear', 'C': 0.010053339249257243, 'gamma': 0.04560640438405416}. Best is trial 2 with value: 0.011685236768802176.
[I 2021-